In [ ]:
#%pip install kagglehub pandas pyarrow datasets
#%pip install "kagglehub[pandas-datasets]"
#%pip install pandas
#%pip install openpyxl
#%pip install sklearn
#%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
import numpy as np

# Set the path to the file you'd like to load
file_path = "Elections Data final.xlsx"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "abeperez/ph-presidential-elections",
  file_path,
)

print(df.head())


/Users/gabdomingo/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/gabdomingo/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/q7/bvbb1cw53mx6lhgk432q4jw80000gn/T/ipykernel_74748/3833996739.py:10: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


   year_of_elections date_of_elections  has_won manner_of_winning  \
0               1981        06/16/1981        0               NaN   
1               1998        05/11/1998        0               NaN   
2               1957        11/12/1957        0               NaN   
3               1961        11/14/1961        0               NaN   
4               1969        11/09/1969        0               NaN   

                                name               party    birthday  \
0  Alejo de los Santos de los Santos  Nacionalista Party  07/17/1911   
1           Alfredo "Fred" Siojo Lim       Liberal Party  12/21/1929   
2                      Alfredo Abcde       Federal Party         NaN   
3                      Alfredo Abcde       Federal Party         NaN   
4                     Angel Comangon         Independent         NaN   

   age_on_elections  sex    city  ... sibling_no alma_mater hi_educ_attain  \
0              69.0  1.0  Bustos  ...        0.0        1.0        College

In [5]:

df.fillna(0)
df.fillna(df.mean(numeric_only=True))
df.iloc[32,2] = 1
print (df.iloc[32,2])

1


In [6]:
# Fill numeric columns with mean
for col in df.select_dtypes(include=['int64', 'float64']).columns:
    df[col] = df[col].fillna(df[col].mean())

# Fill categorical columns with mode
for col in df.select_dtypes(include=['object', 'category']).columns:
    df[col] = df[col].fillna(df[col].mode()[0])

print(df.head())

   year_of_elections date_of_elections  has_won manner_of_winning  \
0               1981        06/16/1981        0   Landslide votes   
1               1998        05/11/1998        0   Landslide votes   
2               1957        11/12/1957        0   Landslide votes   
3               1961        11/14/1961        0   Landslide votes   
4               1969        11/09/1969        0   Landslide votes   

                                name               party    birthday  \
0  Alejo de los Santos de los Santos  Nacionalista Party  07/17/1911   
1           Alfredo "Fred" Siojo Lim       Liberal Party  12/21/1929   
2                      Alfredo Abcde       Federal Party  09/11/1917   
3                      Alfredo Abcde       Federal Party  09/11/1917   
4                     Angel Comangon         Independent  09/11/1917   

   age_on_elections       sex    city  ... sibling_no alma_mater  \
0         69.000000  1.000000  Bustos  ...   0.000000   1.000000   
1         68.000

In [7]:
X = df[["sex","alma_mater","last_college_attended","degree_obtained","hi_educ_attain","last_job","no_of_govt_positions_served", "served_in_gov't", "years_of_service"]]
y = df ["has_won"]
print(X.dtypes)

sex                            float64
alma_mater                     float64
last_college_attended           object
degree_obtained                 object
hi_educ_attain                  object
last_job                        object
no_of_govt_positions_served    float64
served_in_gov't                float64
years_of_service               float64
dtype: object


In [8]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

categorical_cols = X.select_dtypes(include = ['object']).columns
preprocessor = ColumnTransformer( 
    transformers =[('cat', OneHotEncoder(drop='first'), categorical_cols)],
    remainder = 'passthrough'
)


In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

num_cols = X.select_dtypes(include = ['float64', 'int64']).columns

from sklearn.compose import make_column_transformer

#combining categorical encoding and scaling
preprocessor = make_column_transformer(
    (OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), categorical_cols),
    (StandardScaler(), num_cols)
)

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
pipeline.fit(X_train, y_train)

print(pipeline.score(X_test,y_test))


0.9166666666666666


/Users/gabdomingo/Library/Python/3.9/lib/python/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 1, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [13]:
possible_candidate = pd.DataFrame([
    {"sex" : 1, "alma_mater": 2, "last_college_attended": "Secondary", "degree_obtained": "Secondary Education ","hi_educ_attain": " Secondary Education","last_job": "Senator", "no_of_govt_positions_served" : 4, "served_in_gov't" : 1, "years_of_service" : 27.0191780821918},
    {"sex" : 0, "alma_mater": 3, "last_college_attended": "University of the Philippines", "degree_obtained": "Economics","hi_educ_attain": "Doctorate ", "last_job": "Vice President", "no_of_govt_positions_served" : 4, "served_in_gov't" : 1, "years_of_service" : 8.86301369863014},
    {"sex" : 0, "alma_mater": 3, "last_college_attended": "University of the Philippines", "degree_obtained": "Political Science","hi_educ_attain": "Doctorate ", "last_job": "Senator", "no_of_govt_positions_served" : 4, "served_in_gov't" : 1, "years_of_service" : 18.4739726027397},
    {"sex" : 1, "alma_mater": 3, "last_college_attended": "Ateneo De Manila", "degree_obtained": "Economics","hi_educ_attain": "College ", "last_job": "Senator", "no_of_govt_positions_served" : 4, "served_in_gov't" : 1, "years_of_service" : 12},
    {"sex" : 1, "alma_mater": 2, "last_college_attended": "San Beda College", "degree_obtained": "Law","hi_educ_attain": "College ", "last_job": "Mayor", "no_of_govt_positions_served" : 5, "served_in_gov't" : 1, "years_of_service" : 30}

])

predictions = pipeline.predict(possible_candidate)
probs = pipeline.predict_proba(possible_candidate)

for i, (result, prob) in enumerate(zip(predictions, probs)):
    win_percent = prob[1] * 100
    print(f"Candidate {i+1} (Probability: {win_percent:.2f}%)")




Candidate 1 (Probability: 26.81%)
Candidate 2 (Probability: 21.49%)
Candidate 3 (Probability: 7.93%)
Candidate 4 (Probability: 16.89%)
Candidate 5 (Probability: 52.59%)


/Users/gabdomingo/Library/Python/3.9/lib/python/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 1, 2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/gabdomingo/Library/Python/3.9/lib/python/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0, 1, 2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
